In [3]:
import utils
import threading
from datetime import datetime as dt
from playsound import playsound
from configs import API_KEY,BASE_URL,AMOUNT,MIN_PERCENT_REQUIRED,BRL_AMOUNT_TRADE
from biscoint import get_rate_limit, get_balance, get_offer, confirm_offer, get_ticker
from coinsamba import get_offers_coinsamba
import time

In [4]:
#Inicial configs
rate_limit = get_rate_limit()
sleep_time_offers = ((rate_limit["windowMs"] / rate_limit["maxRequests"]) / 1000) * 1.5
no_rush_percent = -0.2 # If the spread is lower than this, wait normal time
initial_balance = get_balance()
last_balance = initial_balance
ticker = get_ticker()
amount_btc_to_trade = utils.btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])

In [5]:
def async_offer(op:str,amount:str,isQuote:bool):
    try:
        response = get_offer(op,amount,isQuote)
        request_orders[op] = response
    except Exception as e:
        print("Erro ao rodar thread",e)
# thread_buy = threading.Thread(target=async_offer,args=('buy',AMOUNT,False))
# thread_sell = threading.Thread(target=async_offer,args=('sell',AMOUNT,False))
# thread_buy.start()
# time.sleep(0.1)
# thread_sell.start()
# thread_buy.join()
# thread_sell.join()

In [6]:
while True:
    start_time = dt.now()
    try:
        request_orders = {'buy':None,'sell':None}


       
        request_orders['buy'] = get_offer('buy',str(amount_btc_to_trade),False)
        request_orders['sell'] = get_offer('sell',str(amount_btc_to_trade),False)

        buy_price = request_orders['buy']['efPrice']
        sell_price = request_orders['sell']['efPrice']
        percent = utils.percent(buy_price, sell_price)

        if percent >= MIN_PERCENT_REQUIRED:
            print(f"Arbitrage oportunity: buy:{buy_price}   sell:{sell_price} ->  {percent}%")
            playsound('beep.wav')
            executed_buy = confirm_offer(buy['offerId'])
            executed_sell = confirm_offer(sell['offerId'])

        print(percent)

        end_time = dt.now()
        seconds_elapsed = (end_time - start_time).total_seconds()
        print(f"Took {seconds_elapsed} seconds")
        print("Wait",sleep_time_offers-percent)
        if percent <= no_rush_percent:
            time.sleep(sleep_time_offers)
        else:
            time.sleep(0.5)
        
    except Exception as e:
        print(e)



-0.882
Took 1.019607 seconds
Wait 3.882
-0.882
Took 0.935965 seconds
Wait 3.882


KeyboardInterrupt: 

In [64]:
sell = get_offers_coinsamba('sell',amount_btc_to_trade)
buy = get_offers_coinsamba('buy',amount_btc_to_trade)

calc_percent = utils.percent(buy[0]['averagePrice'],sell[0]['averagePrice'])
print(calc_percent)

8.107
